## IMPORT

In [1]:
import rasterio
from rasterio.windows import Window
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
from torch import optim
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
#import pandas as pd
import matplotlib.patches as mpatches
from collections import Counter
import matplotlib.pyplot as plt
from rasterio.windows import Window


In [ ]:
# Function to find and visualize windows with specific classes
def visualize_windows_with_classes(raster_path, classes_of_interest, window_size=512, num_windows=3):
    """
    Visualize windows containing specific classes.
    
    Parameters:
    - raster_path: Path to the raster file.
    - classes_of_interest: Set of classes to look for in windows.
    - window_size: Size of the square window to read (in pixels).
    - num_windows: Number of windows to display.
    """
    windows_found = 0
    
    with rasterio.open(raster_path) as src:
        # Raster dimensions
        width = src.width
        height = src.height
        
        # Randomly sample windows
        for row in range(0, height, window_size):
            for col in range(0, width, window_size):
                # Define the window
                window = Window(col_off=col, row_off=row, width=window_size, height=window_size)
                
                # Ensure the window is within bounds
                if col + window_size > width or row + window_size > height:
                    continue
                
                # Read the window
                data = src.read(1, window=window)
                
                # Check if the window contains the desired classes
                if any(cls in data for cls in classes_of_interest):
                    # Plot the window
                    plt.figure(figsize=(6, 6))
                    # The tab20 colormap is one of the built-in colormaps in Matplotlib, 
                    # which provides a set of 20 distinct colors. This can be particularly useful 
                    # for visualizing categorical data with many categories.
                    plt.imshow(data, cmap="tab20", interpolation="none")
                    plt.title(f"Window at col {col}, row {row}")
                    plt.colorbar(label="Classes")
                    plt.show()
                    
                    windows_found += 1
                    
                    # Stop if we've found enough windows
                    if windows_found >= num_windows:
                        return
                else:
                    print(f"Window at col {col}, row {row} does not contain the desired classes.")


In [8]:
# Visualize windows with specific classes
# Classes to visualize (e.g., class 1 and class 2)
# Path to your large TIFF file
raster_path = 'datasets/\\LC_2018_UTM_WGS84_31N_1m_12_Classes.tif'
raster_path = r'D:\tiles_training\tiles_masks\tiffs\tile_24000_0_novalues_blank.tif'
classes_of_interest = {1, 2, 3}
visualize_windows_with_classes(raster_path, classes_of_interest)

In [6]:

# Open the raster
with rasterio.open(raster_path) as src:
    # Initialize an empty Counter to store label frequencies
    label_counts = Counter()
    
    # Read the file in manageable chunks
    for ji, window in src.block_windows(1):  # Iterate over each block in the first band
        data = src.read(1, window=window)    # Read the block
        unique, counts = np.unique(data, return_counts=True)  # Get unique labels and their counts
        label_counts.update(dict(zip(unique, counts)))        # Update the Counter

# Convert to a histogram-friendly format
histogram = dict(sorted(label_counts.items()))

# Display the histogram
print("Histogram of labels:", histogram)


Histogram of labels: {0: 578803026, 1: 65729314, 2: 26829204, 3: 133273474, 4: 65516316, 5: 8500723, 6: 4758221, 7: 2746500, 8: 2810262, 9: 5699709, 10: 603801, 11: 2061981, 12: 2338573, 19: 316612, 41: 12284}
